In [1]:
import requests
import datetime
import pandas as pd
import os

current_date = datetime.datetime.now()

current_month = current_date.month
current_year = current_date.year

months_to_check = [current_month - 2, current_month - 1, current_month]

latest_url = None

for month in months_to_check:
    if month <= 0:
        month = 12
        year = current_year - 1
    elif month > 12:
        month = 1
        year = current_year + 1
    else:
        year = current_year

    url = f"https://rosstat.gov.ru/storage/mediabank/Ipc_mes_{month:02d}-{year}.xlsx"

    response = requests.get(url)

    if response.status_code == 200:
        latest_url = url

if latest_url:
    response = requests.get(latest_url)
    with open('data_raw.xlsx', 'wb') as file:
        file.write(response.content)
    print(f"Файл успешно скачан и сохранен как data_raw.xlsx (из {latest_url})")
else:
    print("Не удалось найти доступный файл. Проверьте подключение к интернету или доступность данных.")

file_path = os.path.join(os.getcwd(), 'data_raw.xlsx')

xls = pd.ExcelFile(file_path)

print(xls.sheet_names)

sheet_name = '01'

df = pd.read_excel(xls, sheet_name=sheet_name, skiprows=3, usecols='A:AI', nrows=13)
df = df.drop([0], axis = 0)
df.rename(columns={df.columns[0]: 'month'}, inplace=True)
df.index.name = None

df = pd.melt(df, id_vars=['month'], var_name='year', value_name='CPI_mom')

from datetime import datetime

month_map = {
    'январь': '01',
    'февраль': '02',
    'март': '03',
    'апрель': '04',
    'май': '05',
    'июнь': '06',
    'июль': '07',
    'август': '08',
    'сентябрь': '09',
    'октябрь': '10',
    'ноябрь': '11',
    'декабрь': '12'
}
df['month'] = df['month'].map(month_map)
df['date'] = '01.' + df['month'] + '.' + df['year'].astype(str)
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df = df.drop(['month', 'year'], axis = 1)
df.set_index('date', inplace = True)
df.index.name = None
df = df[df.index >= '2001-12-01']
df = df.dropna(axis = 0)
df['CPI_base'] = 100

for i in range(1, len(df.index)):
    df.loc[df.index[i], 'CPI_base'] = (df['CPI_mom'][i] * df['CPI_base'][i - 1]) / 100

df["CPI"] = 100
for i in range(13, len(df.index)):
    df.loc[df.index[i], 'CPI'] = ((df['CPI_base'][i] / df['CPI_base'][i - 12]) * 100).round(2)

df = df.drop(['CPI_mom', 'CPI_base'], axis = 1)
df = df[df.index >= '2003-01-01']


Файл успешно скачан и сохранен как data_raw.xlsx (из https://rosstat.gov.ru/storage/mediabank/Ipc_mes_05-2024.xlsx)
['Содержание', '01', '02', '03', '04']
